<h1 style="font-size:2.2em; font-weight:bold; text-align:center;">Exploratory Data Analysis</h1>

This project serves as a **Proof of Concept (PoC)** for identifying potential financial crime within corporate banking transactions. 

The data covers the period from **April 2, 2025** to **December 31, 2025**, based on the assumption that in mid-2025, trade tensions lead to increased abnormal or suspicious transaction activities.

For exploratory data analysis (EDA), we will:

- Integrate dynamic currency conversion using the Riksbanken API

- Explore missing values

- Perform univariate and bivariate analysis: Identify potential features related to fraud

- Conduct descriptive analytics using SQL

- Analyze data distribution and identify outliers




## <h2 style="font-size: 1.6em; font-weight: bold;"> 1. Data Collection </h2>
Import Required Packages and Data

**Importing Pandas, Numpy, Matplotlib, Seaborn, Plotly.express and Warings Library.**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

**Import the CSV Data as Pandas DataFrame**

In [2]:
#Import data from 2 reports and merge them using a left join on `from_account_id`

transaction_df = pd.read_csv('data/transactions.csv')
customer_df = pd.read_csv('data/customers.csv')
sum_statistic_df = pd.read_csv('data/summary statistics.csv')

df = transaction_df.merge(customer_df, on='customer_id', how='left') \
                   .merge(sum_statistic_df, on='customer_id', how='left')

Handle duplicate columns after merging.

In [3]:
dup_cols = []
cols = df.columns.tolist()

for i in range(len(cols)):
    for j in range(i + 1, len(cols)):
        if df[cols[i]].equals(df[cols[j]]):
            dup_cols.append((cols[i], cols[j]))
print("Columns with identical values:")
for c1, c2 in dup_cols:
    print(f"{c1} and {c2}")
    
dropped = set()
kept = set()

for c1, c2 in dup_cols:
    if c2 not in dropped:
        df.drop(columns=c2, inplace=True)
        dropped.add(c2)
        kept.add(c1)     
print("Dropped columns:", list(dropped))
print ("Remaining columns:", list(kept) )

Columns with identical values:
currency_x and currency_y
currency_x and currency
amount and amount_f
country_x and country_y
currency_y and currency
available_balance_x and booked_balance
available_balance_x and available_balance_y
booked_balance and available_balance_y
credit_limit_x and credit_limit_y
Dropped columns: ['available_balance_y', 'currency', 'country_y', 'credit_limit_y', 'amount_f', 'currency_y', 'booked_balance']
Remaining columns: ['available_balance_x', 'credit_limit_x', 'country_x', 'currency_x', 'amount']


In [4]:
df.rename(columns={
    'available_balance_x': 'available_balance',
    'country_x' : 'country',
    'currency_x': 'currency',
    'credit_limit_x' : 'credit_limit'
}, inplace=True)

In [5]:
df.to_csv('data/RAW_transaction_monitoring_merged.csv')

**Dataset Overview**

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   customer_id                      10716 non-null  object 
 1   transaction_id                   10716 non-null  object 
 2   currency                         10716 non-null  object 
 3   booking_date                     10716 non-null  object 
 4   value_date                       10716 non-null  object 
 5   transaction_date                 10716 non-null  object 
 6   payment_date                     10716 non-null  object 
 7   type_description                 10716 non-null  object 
 8   narrative                        10716 non-null  object 
 9   status_x                         10716 non-null  object 
 10  counterparty_name                10716 non-null  object 
 11  amount                           10716 non-null  float64
 12  card_number       

We have a dataset with 10,716 records and 35 columns:

  - Categorical features: 26

  - Numeric features: 9

## <h2 style="font-size: 1.6em; font-weight: bold;"> 2. Data Wrangling </h2>

- Removed Duplicate Entries

- Converted amounts to EUR using exchange rates from the Riskbanken open API

In [7]:
df.duplicated().sum()

107

The data has 200 duplicate records. Drop all duplicate entries.

In [8]:
#Remove Duplicate
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

Converted amounts to EUR to standardize all currencies.

In [9]:
from datetime import datetime, timedelta
import urllib.request, json

# Set up date
single_date = datetime.today() - timedelta(days=5)
date_str = single_date.strftime('%Y-%m-%d')

# Ensure all currency codes are uppercase
currencies = df['currency'].unique()

# Build currency-to-series map
currency_to_series = {
    cur: f'SEK{cur}PMI' if cur != 'SEK' else 'SEK'
    for cur in currencies
}

rate_dict = {}

# Loop to fetch exchange rates
for cur in currencies:
    if cur == 'EUR':
        rate_dict[cur] = 1.0  # EUR to EUR
        continue
    if cur == 'SEK':
        series1 = 'SEK'
    else:
        series1 = currency_to_series[cur]

    series1 = currency_to_series[cur]
    url = f'https://api.riksbank.se/swea/v1/CrossRates/{series1}/SEKEURPMI/{date_str}/{date_str}'

    try:
        req = urllib.request.Request(url)
        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        content = response.read()
        data = json.loads(content)

        # Handle different JSON structures
        if isinstance(data, list) and data:
            rate = data[0]['value']
            rate_dict[cur] = float(rate)
        elif isinstance(data, dict) and 'value' in data:
            rate_dict[cur] = float(data['value'])
        else:
            rate_dict[cur] = 1.0

    except Exception as e:
        print(f"Error fetching rate for {cur}: {e}")
        rate_dict[cur] = 1.0

# Apply conversion to EUR
df['amount_eur'] = df.apply(
    lambda x: float(x['amount']) if x['currency'].upper() == 'EUR'
    else (
        float(x['amount']) / rate_dict.get('SEK', 1.0) if x['currency'].upper() == 'SEK'
        else float(x['amount']) * rate_dict.get(x['currency'].upper(), 1.0)
    ),
    axis=1
)

df[['amount', 'currency','amount_eur']]

Error fetching rate for DKK: HTTP Error 429: Too Many Requests
Error fetching rate for NOK: HTTP Error 429: Too Many Requests


,amount,currency,amount_eur
0,16.31,DKK,16.310000
1,1195.89,NOK,1195.890000
2,5339.23,EUR,5339.230000
3,845.71,EUR,845.710000
4,4808.88,NOK,4808.880000
...,...,...,...
10604,2013.87,SEK,183.957068
10605,11144.82,SEK,1018.024206
10606,152.18,EUR,152.180000
10607,2832.87,DKK,2832.870000


## <h2 style="font-size: 1.6em; font-weight: bold;"> 3. EDA </h2>

- Explored missing values.

- Performed univariate and bivariate analysis.

- Performed descriptive analytics using SQL.

- Analyzed data distribution and identified outliers.


**3.1 Missing values**

In [10]:
missing_pct = round(df.isnull().sum()*100/len(df),2)
missing_column_pct = missing_pct[missing_pct> 1]
missing_column_pct

card_number    80.17
message        80.35
own_message    80.35
dtype: float64

Let’s examine the relationship between missing values in the columns and the target variable `is_fraud` determine the best way to handle them during Feature Engineering.

In [11]:
plt.figure(figsize=(8, 10))

for i, col in enumerate(missing_column_pct.index, 1):
    df_copy = df.copy()
    df_copy[col+'_na'] = np.where(df_copy[col].isnull(), 1, 0)
    
    miss = df_copy.groupby(col+'_na')['is_fraud'].mean().reset_index()
    plt.subplot(2, 3, i)
    sns.barplot(data=miss, x=col+'_na', y='is_fraud', color='darkblue')
    plt.title(f'{col}')
    plt.xlabel('Missing (0 = No, 1 = Yes)')
    plt.ylabel('Mean is_fraud')

plt.tight_layout()
plt.show()

KeyError: 'Column not found: is_fraud'

<Figure size 800x1000 with 0 Axes>

So, only `related_trade_invoice_id` shows a relationship with is_fraud; the others can be dropped.

**3.2 Univariate and Bivariate analysis**

**Numeric Features (Continuous Variable)**

In [ ]:
numerical_feature = df.select_dtypes (include='number')
numerical_feature.describe()

In [ ]:
#As we can see, most of the numeric features are continuous variables. Let’s explore them further.
plt.figure(figsize=(10, 6))
for i, col in enumerate(numerical_feature,1):
      df_copy = df.copy()
      plt.subplot(2, 2, i)
      sns.histplot(df_copy[col], bins=50, kde=True, color='darkblue')
      plt.title(f'Distribution of {col}')
      plt.xlabel(col)
      plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
fraud_counts = (100.00* df['is_fraud'].value_counts()/ len(df)).sort_index()
fraud_df = fraud_counts.reset_index()
fraud_df.columns = ['is_fraud','pct']

plt.figure (figsize=(6,8))
ax = sns.barplot(fraud_df, x='is_fraud', y='pct',color='lightcoral', edgecolor='black')

for index,row in fraud_df.iterrows():
    ax.text (x=index, y=row['pct'] + 0.9, s=f"{row['pct']:.2f}%", ha='center', va='bottom')

plt.show()

In [ ]:
numerical_feature.corr()['is_fraud'] 

`amount_eur` showed the highest correlation with the target variable.

**Temporal Variables (Ex: Datetime Variables)**

Convert  `transaction_date`, `booking_date`, `value_date`, and `payment_date` from object type to datetime format, then transform them into numeric columns.

In [ ]:
df['transaction_day'] = pd.to_datetime(df['transaction_date']).dt.day
df['transaction_day_of_week'] = pd.to_datetime(df['transaction_date']).dt.day_of_week
df['transaction_month'] = pd.to_datetime(df['transaction_date']).dt.month


df['booking_day'] = pd.to_datetime(df['booking_date']).dt.day
df['booking_day_of_week'] = pd.to_datetime(df['booking_day']).dt.day_of_week
df['booking_month'] = pd.to_datetime(df['booking_date']).dt.month


df['value_day'] = pd.to_datetime(df['value_date']).dt.day
df['value_day_of_week'] = pd.to_datetime(df['value_day']).dt.day_of_week
df['value_month'] = pd.to_datetime(df['value_date']).dt.month


df['payment_day'] = pd.to_datetime(df['payment_date']).dt.day
df['payment_day_of_week'] = pd.to_datetime(df['payment_day']).dt.day_of_week
df['payment_month'] = pd.to_datetime(df['payment_date']).dt.month

In [ ]:
transaction_month = df.groupby(df['transaction_month'])['is_fraud'].mean().reset_index()
sns.lineplot(transaction_month, x='transaction_month', y='is_fraud', marker='o', color='darkblue')
plt.title('Fraud Rate by Transaction Month')
plt.xlabel('Transaction Month')
plt.ylabel('Fraud Rate')
plt.show()

In [ ]:
df['transaction_month'].corr(df['is_fraud'])

`transaction_mon` shows a positive Pearson correlation coefficient with `is_fraud`.

**Insight:**

- Fraud rates vary, ranging from about 2.16% to 4.19%.

- August and November show nearly double the fraud rate compared to June.

- The peak in August could be related to increased trade tensions following the expiration of the tariff pause

- Fraud rates hover around 2.5%-3.1%, indicating steady but moderate risk.

**Recommendations:**

- Investigate why months 8 and 11 see spikes in fraud.

- Increase fraud monitoring and controls during high-fraud months.

- Incorporate transaction month as a feature in fraud prediction models.


In [ ]:
Value_month = df.groupby(df['value_month'])['is_fraud'].mean().reset_index()
sns.lineplot(Value_month, x='value_month', y='is_fraud', marker='o', color='darkblue')
plt.title('Fraud Rate by Value Month')
plt.xlabel('Value Month')
plt.ylabel('Fraud Rate')
plt.show()

In [ ]:
df['value_month'].corr(df['is_fraud'])

**Insight**

- Fraud rates begin to rise starting in April, possibly linked to the beginning of trade tensions.

- Peaks occur in August (3.97%) and November (4.02%), aligning with patterns observed in `transaction_month`.

**Recommendations**

- Strengthen fraud detection efforts in August and November, consistent with observed peaks.

- Investigate factors contributing to the increase in fraud beginning in April.

- Integrate both `transaction_mon` and `value_mon` features into predictive models. 


In [ ]:
#Let’s analyze the relationship between is_fraud and the other temporal features.
temporal_features = df[[feature for feature in df.columns if 'mon' in feature] + ['is_fraud']]
tem_corr = temporal_features.corr()['is_fraud']
tem_corr

**Categorical Variables**

In [ ]:
categorical_features = df.select_dtypes(include='object')
categorical_features.columns

In [ ]:
#Identify the number of unique categories in each categorical (object) column.
for feature in categorical_features: 
    count_value = categorical_features[feature].value_counts()
    print (feature,'-', len(count_value),'categories')

**3.3 Descriptive analytics using SQL**

**Purpose:** From a business perspective, define insights: "what the data tells us?". Broken down into two components:

- Overview of Key Metrics: 

        What are the lowest and highest amounts of fraudulent transactions?

        How many fraud vs. non-fraud transactions and their percentage?
       

- Transaction Trends and Behavior:

       How many transactions were sent inside vs. outside the Nordic region?


In [ ]:
!pip install ipython-sql
import sqlite3

# Connect to created in-memory database
con = sqlite3.connect(':memory:')
df.to_sql ('RAW_nordic_transactions_with_fraud', con, index=False)

*What are the lowest and highest amounts of fraudulent transactions?*

In [ ]:
query = '''
Select 'lowest_transaction' as category, from_account_name, counterparty_country,currency, amount_eur, is_fraud 
From 
(Select *
From RAW_nordic_transactions_with_fraud
Where is_fraud = '1'
Order by amount_eur asc
Limit 1)

UNION ALL
Select 'highest_transaction' as category, from_account_name, counterparty_country,currency, amount_eur, is_fraud 
From 
(Select *
From RAW_nordic_transactions_with_fraud
Where is_fraud = '1'
Order by amount_eur desc
Limit 1)

'''

df_result = pd.read_sql_query(query, con)
df_result

**Insight:**

- Fraudulent amounts span from low (small test transfers) to high (attempts to extract large sums).
- Both the lowest and highest transaction amounts were sent to high-risk countries, based on sanctions and the 2025 trade tension tariff threats.

*How many fraud vs. non-fraud transactions and their percentage?*

In [ ]:
query = '''
Select 
is_fraud, count (*) as transaction_count,
Round (100.00 * COUNT(*) / 10116,2) as percentage
From RAW_nordic_transactions_with_fraud
Group by is_fraud
'''

df_result = pd.read_sql_query (query, con)
df_result

**Insight:**   

- Fraud is likely a small percentage (2.97%) compared to 97.03% of non-fraudulent transactions.

*How many transactions were sent inside vs. outside the Nordic region?*

In [ ]:
query = '''
Select region, count (*) as transaction_count, 
Round (100* count (*)/ 10116, 2) as percentage
From 
(Select *,
Case when counterparty_country IN ('SE', 'NO', 'FI', 'DK', 'IS') Then 'Nordic' else 'non_Nordic' End as region
From RAW_nordic_transactions_with_fraud
)
Group by region
'''
df_result = pd.read_sql_query (query, con)
df_result

**Insight:**  

- Transactions are nearly evenly split: 47% from Nordic, 53% from non-Nordic.

- There are slightly more transactions coming from non-Nordic regions than Nordic ones.

**Suggestion:** 

- Segment fraud prevention strategies by region.

**3.4 Outliers and Data Distribution**

As we discovered, most numeric features are skewed. Therefore, we will apply a log transformation to better identify outliers.

In [ ]:
from numpy import log
plt.figure(figsize=(10,10))
for i, col in enumerate(numerical_feature,1):
    df_copy=df.copy()
    if 0 in df_copy[col].unique():
        pass
    else:
        plt.subplot(2, 2, i)
        df_copy[col] = np.log(df_copy[col])
        sns.boxplot(df_copy, y=col)
        plt.title(f'{col}')
plt.show()

As we observed, most of the numeric features contain outliers.

In [ ]:
df.head().T

In [ ]:
df.to_csv('data/EDA_nordic_transactions_with_fraud.csv', index=False)